installing the libraries

In [ ]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install -U git+https://github.com/huggingface/trl
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 10.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/trl to /tmp/pip-req-build-lbfn9n8w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl /tmp/pip-req-build-lbfn9n8w
  Resolved https://github.com/huggingface/trl to commit 28bdb6a3736b09f8bad7961f16852d052c74ae04
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 2.2 MB/s eta 0:00:00
  Created wheel for trl: filename=trl-0.7.5.dev0-py3-none-any.whl size=134112 s

importing the libraries

In [ ]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments,BitsAndBytesConfig, HfArgumentParser
from dataclasses import dataclass, field
from typing import Optional
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from trl import RewardConfig, RewardTrainer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


loading the model to train reward model

In [ ]:
quantization_config = BitsAndBytesConfig(
      load_in_4bit=True
)
device_map = {"": Accelerator().local_process_index}

model = AutoModelForSequenceClassification.from_pretrained(
    "facebook/opt-350m",
    quantization_config=quantization_config,
    device_map=device_map,
    trust_remote_code=True,
    num_labels=1,
)
tokenizer1 = AutoTokenizer.from_pretrained("facebook/opt-350m")

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

importing the dataset and pre-processing it

In [ ]:
# train_dataset = load_dataset("Anthropic/hh-rlhf",split="train")
# train_dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 160800
})

In [ ]:
train_dataset = pd.read_csv("/content/accepted_rejected.csv",header=None)
train_dataset = train_dataset.iloc[:,[1,2]]
train_dataset.rename(columns = {1:'chosen',2:'rejected'}, inplace = True)
train_dataset

,chosen,rejected
0,Question : ‘’The wealth of Nation” Considered ...,### Question: ‘’The wealth of Nation” ...
1,Question : Business Economics is:Option A : N...,### Question: Business Economics is: ...
2,Question : The Central Economic Problem is:Opt...,### Question: The Central Economic Pro...
3,Question : The ‘communist Manifesto’ was publi...,### Question: The ‘communist Manifesto...
4,Question : Which of the following is not withi...,### Question: Which of the following i...
...,...,...
233,Question : Which of the following is not withi...,### Question: Which of the following i...
234,Question : What is consolidation?Option A : I...,### Question: What is consolidation? ...
235,Question : _________________________ is the si...,### Question: ________________________...
236,Question : _________is a combination of severa...,### Question: _________is a combinatio...


In [ ]:
train_dataset = Dataset.from_pandas(train_dataset)
train_dataset

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 238
})

In [ ]:
def preprocess_function(examples):
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(examples["chosen"], examples["rejected"]):
        tokenized_chosen = tokenizer(chosen, truncation=True)
        tokenized_rejected = tokenizer(rejected, truncation=True)

        new_examples["input_ids_chosen"].append(tokenized_chosen["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_chosen["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_rejected["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_rejected["attention_mask"])

    return new_examples

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
)

train_dataset = train_dataset.filter(
    lambda x: len(x["input_ids_chosen"]) <= 512
    and len(x["input_ids_rejected"]) <= 512
)

Map (num_proc=4):   0%|          | 0/238 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Filter:   0%|          | 0/238 [00:00<?, ? examples/s]

training the model

In [ ]:
training_args = RewardConfig(
    output_dir="output",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    learning_rate=1.41e-5,
    # report_to="wandb" if script_args.log_with == "wandb" else "tensorboard",
    remove_unused_columns=False,
    optim="adamw_torch",
    logging_steps=1,
    # evaluation_strategy="steps" if script_args.eval_split != "none" else "no",
    max_length=512,
)
peft_config = LoraConfig(r=16, lora_alpha=16, bias="none", task_type="SEQ_CLS", modules_to_save=["scores"])

In [ ]:
trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
    peft_config=peft_config,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,2.189100
2,0.428200
3,1.103200
4,0.705000
5,1.464700
6,1.769700
7,1.006300
8,1.096100
9,0.113500
10,1.835800


TrainOutput(global_step=237, training_loss=0.39390435872966906, metrics={'train_runtime': 125.2516, 'train_samples_per_second': 1.892, 'train_steps_per_second': 1.892, 'total_flos': 0.0, 'train_loss': 0.39390435872966906, 'epoch': 1.0})

In [ ]:
def get_score(model, tokenizer, prompt, response):
    # Tokenize the input sequences
    inputs = tokenizer.encode_plus(prompt, response, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

    # Perform forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the logits
    logits = outputs.logits

    return logits.item()

In [ ]:
prompt = "What is Depreciation"
example_less_pref_response = " Human: What are some cuss words in english? Assistant: Here’s an incomplete list. Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch, Human: What's your favorite one? Assistant: Ass."  # Insert the actual response here
example_preferred_response = "Human: What are some cuss words in english? Assistant: Here’s an incomplete list. Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch, Human: What's your favorite one? Assistant: I haven't even thought about it."  # Insert the actual response here

# Get the score for the less preferred response
score_less_pref = get_score(model, tokenizer, prompt, example_less_pref_response)
print("Score for less preferred response:", score_less_pref)
# >> -3.915163993835449

# Get the score for the preferred response
score_preferred = get_score(model, tokenizer, prompt, example_preferred_response)
print("Score for preferred response:", score_preferred)
# # >> 7.460323333740234

Score for less preferred response: 2.276484727859497
Score for preferred response: 3.1755709648132324


In [ ]:
prompt = "What is Depreciation"
example_less_pref_response = "    ### Question:     ‘’The wealth of Nation” Considered as the first modern work of Economics was written by :     ### Option A:     Marx     ### Option B:     Adam-Smith     ### Option C:     Lipsey     ### Option D:     Allen     ### Correct Ans:     A     ### Above answer is correct because:      Marx introduced the concept of Valuation in Economics, which later became the basis for the Marxian Class- Structure. Allen is a pioneer in the field of micro-economics, so his work is not necessarily the first in Economics. Lipsey is a pioneer in the field of macroeconomics, so his work is also not necessarily the first in Macroeconomics. Therefore, the correct answer is (A) Marx. The Wealth of the Nation is a broad subject area, and Marx's work is only one of the many contributions made in this field. Therefore, the correct answer is (A) Marx. The Wealth of the Nation is a broad subject area, and Marx's work is only one of the many contributions made in this field. The Wealth of the Nation is a broad subject area, and Marx's work is only "
example_preferred_response = "Question : ‘’The wealth of Nation” Considered as the first modern work of Economics was written by :Option A  : MarxOption B  : Adam-SmithOption C  : LipseyOption D  : AllenCorrect Ans  : (b) Adam-Smithexplanation 	  : The correct answer to the question is (b) Adam-Smith. The wealth of Nations, considered the first modern work of economics, was written by Adam Smith."
# Get the score for the less preferred response
score_less_pref = get_score(model, tokenizer, prompt, example_less_pref_response)
print("Score for less preferred response:", score_less_pref)
# >> -3.915163993835449

# Get the score for the preferred response
score_preferred = get_score(model, tokenizer, prompt, example_preferred_response)
print("Score for preferred response:", score_preferred)
# # >> 7.460323333740234

Score for less preferred response: 1.26975679397583
Score for preferred response: 3.6761422157287598
